In [137]:
# http 요청 응답, html 구문 분석
import requests
from bs4 import BeautifulSoup
import re

In [138]:
# 셀레니움 브라우저의 동작을 자동화하는 패키지
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [139]:
# 데이터 분석 패키지
import pandas as pd

In [178]:
etf_name = input('ETF상품명을 입력하세요')
print(etf_name)

ETF상품명을 입력하세요 KODEX 단기채권PLUS


KODEX 단기채권PLUS


In [140]:
url = 'https://finance.naver.com/sise/etf.naver'
# res = requests.get(url)
# res.status_code

In [ ]:
# 동적으로 요소가 생성되므로 찾고자 하는 정보를 찾을 수 없음
pattern = re.compile(r'')
pattern.search(res.text)

In [141]:
# # 브라우저 종료 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)

# # 웹 드라이버를 이용한 브라우저 제어
driver = webdriver.Chrome(options=chrome_options)

In [142]:
driver.get(url)
# 페이지가 모두 로드될때 까지 최대 2초대기

# Selenium을 사용하여 웹 페이지에서 데이터를 수집할 때, 요청 사이에 텀(지연 시간)을 두는 것은 좋은 습관입니다. 이는 여러 가지 이유로 유용합니다:
# 이유
#  서버에 대한 부담 감소:
#  요청을 너무 빠르게 보내면 서버에 과부하를 일으킬 수 있습니다. 이는 서버가 IP를 차단하거나 요청을 제한할 수 있는 원인이 될 수 있습니다.
#  동적 콘텐츠 로딩:
#  웹 페이지의 콘텐츠가 JavaScript로 동적으로 로드되는 경우, 페이지가 완전히 로드될 때까지 기다리지 않으면 필요한 데이터를 가져오지 못할 수 있습니다.
#  디버깅:
#  코드의 실행 속도가 너무 빠르면, 디버깅이 어려울 수 있습니다. 텀을 두면 각 단계의 결과를 쉽게 확인할 수 있습니다.

driver.implicitly_wait(2)

In [143]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [167]:
# etf_name = 'KODEX 단기채권PLUS'

In [168]:
etf_name

'KODEX 단기채권PLUS'

In [179]:
detail_url = 'https://finance.naver.com' + soup.select_one('#etfItemTable').find('a',string=etf_name).attrs['href']

In [180]:
detail_url

'https://finance.naver.com/item/main.naver?code=214980'

In [171]:
driver.get(detail_url)

In [172]:
trs = driver.find_elements(By.CSS_SELECTOR,'#content > div.section.etf_asset > table > tbody > tr')
len(trs)

17

In [173]:
filtered_rows = []

for tr in trs[1:]:
    # tr 내의 모든 td 요소 찾기
    tds = tr.find_elements(By.CSS_SELECTOR, 'td')

    # 'blank' 클래스가 포함된 td가 있거나, td가 없거나, 클래스명이 'division_line'인 td가 존재하는 경우 제외
    if all ('blank' not in td.get_attribute('class') and 
            'division_line' not in td.get_attribute('class') for td in tds) :
        # print([td.text for td in tr.find_elements(By.CSS_SELECTOR,'td')])
  
        filtered_rows.append([td.text for td in tr.find_elements(By.CSS_SELECTOR,'td')])

print(len(filtered_rows))
print(filtered_rows)

10
[['국민은행4408이표일(03)1.5-22', '-', '-'], ['농업금융채권(은행)2023-06이2Y-B', '-', '-'], ['부산은행2024-01이1A-22', '-', '-'], ['부산은행2024-04이1A-05', '-', '-'], ['부산은행2024-07이1A-03', '-', '-'], ['부산은행2024-12이11M-12', '-', '-'], ['부산은행2024-12이1A-10', '-', '-'], ['산업금융채권24신이0106-0312-1', '-', '-'], ['산업금융채권24신이0106-0823-2', '-', '-'], ['신한금융지주146-1', '-', '-']]


In [174]:
columns = [td.text for td in trs[0].find_elements(By.CSS_SELECTOR,'th')]
columns

['구성종목(구성자산)', '주식수(계약수)', '구성비중']

In [175]:
df = pd.DataFrame(filtered_rows,columns=columns)

In [176]:
df

,구성종목(구성자산),주식수(계약수),구성비중
0,국민은행4408이표일(03)1.5-22,-,-
1,농업금융채권(은행)2023-06이2Y-B,-,-
2,부산은행2024-01이1A-22,-,-
3,부산은행2024-04이1A-05,-,-
4,부산은행2024-07이1A-03,-,-
5,부산은행2024-12이11M-12,-,-
6,부산은행2024-12이1A-10,-,-
7,산업금융채권24신이0106-0312-1,-,-
8,산업금융채권24신이0106-0823-2,-,-
9,신한금융지주146-1,-,-


In [ ]:
driver.quit()